## **Import Required Libraries**

In [ ]:
# Import pandas
import pandas as pd

# Import libraries for web scraping
import requests
from bs4 import BeautifulSoup

# Import wikipedia
import wikipedia

# Import matplotlib for plotting
import matplotlib.pyplot as plt

# Import json to load config file
import json

## **Creates US Election Data**

### **Import Raw Data**

In [ ]:
# Load the file path from the config.json file
with open('config.json', 'r') as file:
    config = json.load(file)

# Access the input directory path
inp_dir = config['input_data_dir']

# Import recent election results raw data
us_re_raw = pd.read_csv(f"{inp_dir}US_Election_Results.csv")

### **Clean Data**

In [ ]:
# Create copy of data to modify
us_re = us_re_raw.copy()

# Convert votes field to numeric
us_re["Votes"] = us_re["Votes"].str.replace(",","")
us_re["Votes"] = us_re["Votes"].apply(pd.to_numeric)

# Remove "wins" from Candidate field
us_re["Candidate"] = us_re["Candidate"].str.replace(" wins","")

# Remove "(incumbent)" from party field
us_re["Party"] = us_re["Party"].str.replace(" (incumbent)","")

### **Add Alignment of Each Party**

In [ ]:
# Create mapping from party to political alignment (using Wikipedia)
party_alignment = {
    "Republican": "Center-right to right-wing",
    "Democrat": "Centre-left",
    "Green": "Left-wing",
    #"Robert F. Kennedy Jr": 
    "Libertarian": "Center-right",
    "Peace and Freedom": "Left-wing",
    "Unity": "Centre",
    "Constitution": "Far-right",
    "American Solidarity": "Centre",
    "Socialist Equality": "Far-left",
    "Socialist Workers Party": "Far-left",
    "Prohibition": "Centre",
    "Independent": None
}

# Create numerical mapping of Alignment
#   ranges from -6 (Far-Left) to 6 (Far-Right)
alignment_mapping = {
    "Centre-left": -2,
    "Centre-right": 2,
    "Centre": 0,
    "Right-wing": 4,
    "Left-wing": -4,
    "Centre to centre-left": -1,
    "Centre to centre-right": 1,
    "Centre-right to right-wing": 3,
    "Center-left": -2,
    "Big tent": 0,
    "Centre-left to left-wing": -3,
    "Far-left": -6,
    "Centre-rightFactions:Right-wing": 3,
    "CentreFactions:Left-wing and right-wing": 0,
    "\nMajority:\nRight-wing": 4,
    "Right-wing to far-right": 5,
    "Center-right": 2,
    "Center-right to right-wing": 3,
    "Far-right": 6,
    None: "No Data",        # PLACEHOLDER: CHANGE!
}

# Map Party to alignment
us_re["Alignment"] = us_re["Party"].map(party_alignment)

# Map alignment to numerical alignment
us_re["Alignment Score"] = us_re["Alignment"].map(alignment_mapping)

# Check that all valid parties have been mapped
print(f"Number of unmapped alignments: {len(us_re[(~us_re['Alignment'].isna()) & (us_re['Alignment Score'].isna())])}")

# Preview data
us_re.head()

### **Create Spectrum Results Data**

In [ ]:
# Get vote count by alignment
align_votes = us_re.groupby(['Alignment Score'])['Votes'].sum().sort_values(ascending=False).reset_index()

# Remove "No Data" Alignment
align_votes = align_votes[align_votes["Alignment Score"] != 'No Data']

# Create % of total column
align_votes['In-Scope Vote Prop'] = (align_votes['Votes'] / align_votes['Votes'].sum())
align_votes['In-Scope Vote %'] = (align_votes['Votes'] / align_votes['Votes'].sum()) * 100
align_votes['In-Scope Vote %'] = align_votes['In-Scope Vote %'].round(2)

# View data
align_votes

## **Data Visualisation**

### **Plot the Data**

In [ ]:
# Create the bar chart
plt.figure(figsize=(10, 5))
plt.bar(align_votes['Alignment Score'], align_votes['In-Scope Vote %'], color='skyblue')

# Add labels and title
plt.xlabel('Alignment Score')
plt.ylabel('In-Scope Vote %')
plt.title(r'Vote % by Alignment Score')
plt.show()

### **Give Alignment Score**

In [ ]:
# Give alignment score for US 2024 election
align_score = align_votes["Alignment Score"].dot(align_votes["In-Scope Vote Prop"])
print(f"Alignment score of the 2024 US Election: {align_score}")

## **Export Data**

In [ ]:
# Access the input directory path
out_dir = config['inter_outputs_dir']

align_votes.to_csv(f"{out_dir}US_New.csv", index=False)